In [2]:
feedback_produto="""Adquiri recentemente o smartphone XYZ e estou extremamente satisfeito com a e
xperiência até agora. O desempenho é rápido e fluido, mesmo com vários aplicativos abertos ao mesmo
tempo. A qualidade da câmera me surpreendeu, especialmente em ambientes com pouca luz — as fotos saem 
nítidas e com cores vibrantes. Outro ponto positivo é a duração da bateria, que aguenta tranquilamente um
dia inteiro de uso intenso. O design também merece destaque: elegante, leve e com ótima pegada. A interface
do sistema é intuitiva e cheia de recursos úteis. Recomendo fortemente para quem busca um celular com ótimo 
custo-benefício, desempenho confiável e recursos modernos!"""

In [3]:
from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template("""
    Para o texto a seguir, extraia as seguintes informações:
    Produto: nome do produto mensionado no texto
    caracteristicas positivas: Liste toda as caracteristicas positivas mencionadas sobre o produto                                               
    caracteristicas negativas: Liste toda as caracteristicas negativas mencionadas sobre o produto
    recomendação: o cliente recomenda o produto?

    texto: {review}

    retorne a resposta no formato json                                               
""")

In [4]:
print(review_template.format_messages(review = feedback_produto))

[HumanMessage(content='\n    Para o texto a seguir, extraia as seguintes informações:\n    Produto: nome do produto mensionado no texto\n    caracteristicas positivas: Liste toda as caracteristicas positivas mencionadas sobre o produto                                               \n    caracteristicas negativas: Liste toda as caracteristicas negativas mencionadas sobre o produto\n    recomendação: o cliente recomenda o produto?\n\n    texto: Adquiri recentemente o smartphone XYZ e estou extremamente satisfeito com a e\nxperiência até agora. O desempenho é rápido e fluido, mesmo com vários aplicativos abertos ao mesmo\ntempo. A qualidade da câmera me surpreendeu, especialmente em ambientes com pouca luz — as fotos saem \nnítidas e com cores vibrantes. Outro ponto positivo é a duração da bateria, que aguenta tranquilamente um\ndia inteiro de uso intenso. O design também merece destaque: elegante, leve e com ótima pegada. A interface\ndo sistema é intuitiva e cheia de recursos úteis. Rec

In [6]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()
resposta = chat.invoke(review_template.format_messages(review = feedback_produto))

In [7]:
resposta.content

'{\n    "Produto": "smartphone XYZ",\n    "caracteristicas positivas": [\n        "Desempenho rápido e fluido",\n        "Qualidade da câmera surpreendente, especialmente em ambientes com pouca luz",\n        "Duração da bateria aguenta um dia inteiro de uso intenso",\n        "Design elegante, leve e com ótima pegada",\n        "Interface do sistema intuitiva e cheia de recursos úteis"\n    ],\n    "caracteristicas negativas": [],\n    "recomendação": "Sim"\n}'

In [8]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [9]:
schema_produto = ResponseSchema(
    name="Produto",
    type="string",
    description="Nome do produto mencionado no texto"
)

schema_positivas = ResponseSchema(
    name="caracteristicas_positivas",
    type="list",
    description="Liste todas as caracteristicas positivas mencionadas sobre o produti"
)

schema_negativas = ResponseSchema(
    name="caracteristicas_negativas",
    type="list",
    description="Liste todas as caracteristicas negativas mencionadas sobre o produti"
)

schema_recomendacao = ResponseSchema(
    name="recomendacao",
    type="bool",
    description="O cliente recomenda o produto? "
)

In [11]:
response_schema = [schema_produto, schema_recomendacao, schema_positivas, schema_negativas]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
schema_formatado = output_parser.get_format_instructions()

In [12]:
print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Produto": string  // Nome do produto mencionado no texto
	"recomendacao": bool  // O cliente recomenda o produto? 
	"caracteristicas_positivas": list  // Liste todas as caracteristicas positivas mencionadas sobre o produti
	"caracteristicas_negativas": list  // Liste todas as caracteristicas negativas mencionadas sobre o produti
}
```


In [14]:
review_template2 = ChatPromptTemplate.from_template("""
Para o texto a seguir extraia as seguintes informações:
produto, caracteristicas positivas e negativas, recomendação
                                                    
texto: {review}
{schema}
""", partial_variables={"schema":schema_formatado}
)

In [15]:
print(review_template2.format_messages(review=feedback_produto))

[HumanMessage(content='\nPara o texto a seguir extraia as seguintes informações:\nproduto, caracteristicas positivas e negativas, recomendação\n\ntexto: Adquiri recentemente o smartphone XYZ e estou extremamente satisfeito com a e\nxperiência até agora. O desempenho é rápido e fluido, mesmo com vários aplicativos abertos ao mesmo\ntempo. A qualidade da câmera me surpreendeu, especialmente em ambientes com pouca luz — as fotos saem \nnítidas e com cores vibrantes. Outro ponto positivo é a duração da bateria, que aguenta tranquilamente um\ndia inteiro de uso intenso. O design também merece destaque: elegante, leve e com ótima pegada. A interface\ndo sistema é intuitiva e cheia de recursos úteis. Recomendo fortemente para quem busca um celular com ótimo \ncusto-benefício, desempenho confiável e recursos modernos!\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"Produto": string  // No

In [17]:
resposta = chat.invoke(review_template2.format_messages(review = feedback_produto))
resposta.content

'```json\n{\n\t"Produto": "smartphone XYZ",\n\t"recomendacao": true,\n\t"caracteristicas_positivas": ["Desempenho rápido e fluido", "Qualidade da câmera em ambientes com pouca luz", "Duração da bateria para um dia inteiro de uso intenso", "Design elegante e leve", "Interface intuitiva e repleta de recursos úteis"],\n\t"caracteristicas_negativas": []\n}\n```'

In [18]:
resposta_json = output_parser.parse(resposta.content)

In [22]:
resposta_json["Produto"]

'smartphone XYZ'